In [1]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pymc3 as pm
from pymc3 import Model,Lognormal,Constant,Dirichlet,Categorical
from pymc3 import NUTS, sample, CategoricalGibbsMetropolis,find_MAP
from scipy import optimize
from pymc3 import traceplot
from pymc3 import summary
import theano.tensor as tt
import theano as T

df = pd.read_csv("../data/data-lda.txt")

n_person = len(df["PersonID"].unique())
n_item =120
K = 6
IDs = df.values[:,0].astype(np.int32)-1
Items = df.values[:,1].astype(np.int32)-1

#shape = クラスの数の確率変数に、クラスの値を取るデータ数次元のベクトルを入れる操作がありますが
#その詳細な説明は(https://pymc-devs.github.io/pymc3/notebooks/GLM-hierarchical.html)参照

basic_model = Model()
with basic_model:
    #事前分布[50,6]
    theta = Dirichlet('p_theta', a=(1.0/K)*np.ones(K),shape=(n_person,K))
    #事前分布[6,112]
    phi = Dirichlet('p_phi', a=(1.0/n_item)*np.ones(n_item), shape=(K,n_item))
    
    #likelihood
    #データ数 x 各データのカテゴリー確率ベクトル [1117,6]
    theta = theta[IDs,:] 
    #データ数 x 各IDに対するアイテム確率ベクトル [1117,112]
    person_to_item = tt.dot(theta, phi)
    
    H = Categorical("tes", p=person_to_item, shape=(1117), observed=Items)

    #サンプリング
    step = CategoricalGibbsMetropolis(vars=[H])
    trace = sample(12, init=None, step=step)
    summary(trace)

Assigned NUTS to p_theta_stickbreaking_
Assigned NUTS to p_phi_stickbreaking_
100%|██████████| 12/12 [00:51<00:00,  9.33s/it]


p_theta:

  Mean             SD               MC Error         95% HPD interval
  -------------------------------------------------------------------
  ..............................[0, :]...............................
  0.276            0.080            0.023            [0.167, 0.405]
  0.030            0.036            0.010            [0.000, 0.093]
  0.162            0.186            0.054            [0.000, 0.499]
  0.076            0.146            0.042            [0.004, 0.554]
  0.298            0.181            0.052            [0.077, 0.569]
  0.158            0.078            0.022            [0.025, 0.343]
  ..............................[1, :]...............................
  0.412            0.200            0.058            [0.176, 0.812]
  0.116            0.110            0.032            [0.003, 0.304]
  0.029            0.039            0.011            [0.000, 0.096]
  0.100            0.099            0.029            [0.000, 0.305]
  0.199            0.102     